In [9]:
import numpy as np
import pandas as pd
import matplotlib as plt
import sqlite3
from pathlib import Path

In [43]:
input_csv = 'COL.csv'
output_db = 'precios_de_cierre.db'

def extraccion(input_csv):
    print(f'Lectura CSV:', input_csv)
    data = pd.read_csv(input_csv, delimiter = ';', encoding = 'UTF-8')
    print('\nDatos Originales')
    print(data.head())
    return data
    
data = extraccion(input_csv)
data

Lectura CSV: COL.csv

Datos Originales
                       Price   Close     Close.1 Close.2 Dividends  \
0                     Ticker  BVC.CL  MINEROS.CL  PEI.CL    BVC.CL   
1                   Datetime     NaN         NaN     NaN       NaN   
2  2024-08-02 13:30:00+00:00   11180        3350   70020       NaN   
3  2024-08-02 14:30:00+00:00     NaN        3200   69900       NaN   
4  2024-08-02 15:30:00+00:00     NaN        3200   69020       NaN   

  Dividends.1 Dividends.2    High      High.1  High.2  ...   Low.2    Open  \
0  MINEROS.CL      PEI.CL  BVC.CL  MINEROS.CL  PEI.CL  ...  PEI.CL  BVC.CL   
1         NaN         NaN     NaN         NaN     NaN  ...     NaN     NaN   
2           0           0   11180        3350   70440  ...   70020   11180   
3           0           0     NaN        3345   69900  ...   69900     NaN   
4           0           0     NaN        3250   69940  ...   69020     NaN   

       Open.1  Open.2 Stock Splits Stock Splits.1 Stock Splits.2  Volum

,Price,Close,Close.1,Close.2,Dividends,Dividends.1,Dividends.2,High,High.1,High.2,...,Low.2,Open,Open.1,Open.2,Stock Splits,Stock Splits.1,Stock Splits.2,Volume,Volume.1,Volume.2
0,Ticker,BVC.CL,MINEROS.CL,PEI.CL,BVC.CL,MINEROS.CL,PEI.CL,BVC.CL,MINEROS.CL,PEI.CL,...,PEI.CL,BVC.CL,MINEROS.CL,PEI.CL,BVC.CL,MINEROS.CL,PEI.CL,BVC.CL,MINEROS.CL,PEI.CL
1,Datetime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-02 13:30:00+00:00,11180,3350,70020,NaN,0,0,11180,3350,70440,...,70020,11180,3350,70440,NaN,0,0,NaN,0,0
3,2024-08-02 14:30:00+00:00,NaN,3200,69900,NaN,0,0,NaN,3345,69900,...,69900,NaN,3345,69900,NaN,0,0,NaN,320079,194
4,2024-08-02 15:30:00+00:00,NaN,3200,69020,NaN,0,0,NaN,3250,69940,...,69020,NaN,3250,69940,NaN,0,0,NaN,360144,1718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,2025-08-01 15:30:00+00:00,NaN,7270,70280,NaN,0,0,NaN,7270,70280,...,70280,NaN,7270,70280,NaN,0,0,NaN,12808,1351
1367,2025-08-01 16:30:00+00:00,NaN,7280,70180,NaN,0,0,NaN,7280,70180,...,70100,NaN,7280,70100,NaN,0,0,NaN,4455,3983
1368,2025-08-01 17:30:00+00:00,NaN,7440,70180,NaN,0,0,NaN,7440,70180,...,70180,NaN,7310,70180,NaN,0,0,NaN,12588,533
1369,2025-08-01 18:30:00+00:00,NaN,7550,NaN,NaN,0,NaN,NaN,7550,NaN,...,NaN,NaN,7500,NaN,NaN,0,NaN,NaN,12399,NaN


In [86]:
nombres = ['bvc', 'mineros', 'pei']

def transformacion(input_csv, numero_de_activos: int, nombres_de_activos: list):
    col_names = ['Close', 'High', 'Low', 'Open']
    data = extraccion(input_csv)
    data = data.set_index('Price')
    
    dfs = [] 

    for i in range(len(nombres_de_activos)):
        df_activo = data.iloc[:, [0 + i, 6 + i, 9 + i, 12 + i]]
        df_activo.columns = col_names
        df_activo = df_activo.drop(['Datetime', 'Ticker'], axis=0, errors='ignore')
        df_activo = df_activo.fillna(method='ffill')
        df_activo = df_activo.astype(float)
        df_activo['Return'] = df_activo['Close'].pct_change().fillna(0)
        dfs.append(df_activo)  

    return dfs

In [87]:
def carga(dfs):
    conn = sqlite3.connect(output_db)

    for df, nombre in zip(dfs, nombres):
        df.to_sql(nombre, conn, index=False)

In [88]:
carga(dfs)